##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 迁移 TensorBoard：TensorFlow 的呈现工具包

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/guide/migrate/tensorboard"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/migrate/tensorboard.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/migrate/tensorboard.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/guide/migrate/tensorboard.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>


[TensorBoard](https://tensorflow.google.cn/tensorboard) 是一个内置工具，用于在 TensorFlow 中提供测量和呈现。可以在 TensorBoard 中跟踪和显示准确率和损失等常见的机器学习实验指标。TensorBoard 与 TensorFlow 1 和 2 代码兼容。

在 TensorFlow 1 中，`tf.estimator.Estimator` 默认为 TensorBoard 保存摘要。相比之下，在 TensorFlow 2 中，可以使用 `tf.keras.callbacks.TensorBoard` <a href="https://keras.io/api/callbacks/" class="external">回调</a>保存摘要。

本指南首先演示了如何在 TensorFlow 1 中将 TensorBoard 与 Estimator 一起使用，然后演示了如何在 TensorFlow 2 中执行等效的过程。

### 安装

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import tempfile
import numpy as np
import datetime
%load_ext tensorboard

In [ ]:
mnist = tf.keras.datasets.mnist # The MNIST dataset.

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### TensorFlow 1：TensorBoard 与 tf.estimator 一起使用

在此 TensorFlow 1 示例中，您将实例化 `tf.estimator.DNNClassifier`，在 MNIST 数据集上对其进行训练和评估，并使用 TensorBoard 显示指标：

In [ ]:
%reload_ext tensorboard

feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.1,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

In [ ]:
%tensorboard --logdir {classifier.model_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF1.png"/> -->

### TensorFlow 2: TensorBoard 与 Keras 回调和 Model.fit 一起使用

在此 TensorFlow 2 示例中，您将使用 `tf.keras.callbacks.TensorBoard` 回调创建和存储日志并训练模型。回调跟踪每个周期的准确率和损失。它会被传递给 `callbacks` 列表中的 `Model.fit`。

In [ ]:
%reload_ext tensorboard

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              steps_per_execution=10)

log_dir = tempfile.mkdtemp()
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=log_dir,
  histogram_freq=1) # Enable histogram computation with each epoch.

model.fit(x=x_train,
          y=y_train,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir {tensorboard_callback.log_dir}

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_TF2.png"/> -->


# 后续步骤

- 在[使用入门](https://tensorflow.google.cn/tensorboard/get_started)指南中详细了解 TensorBoard。
- 对于较低级别的 API，请参阅 [tf.summary 迁移到 TensorFlow 2](https://tensorflow.google.cn/tensorboard/migrate) 指南。